In [ ]:

import numpy as np
import matplotlib.pyplot as plt
from scipy.linalg import eigvalsh
from sklearn.metrics import roc_curve, auc
import pickle

%matplotlib inline
plt.style.use('seaborn-darkgrid')
np.random.seed(42)


In [ ]:

# Simulate loading φ⁰_v10 (fallback random matrix)
phi0_v10 = np.random.standard_normal((8,8))
phi0_v10 = (phi0_v10 + phi0_v10.T)/2.0

# Boost Planarity (row 5) and Curvature (row 6) by 30%
phi0_v11 = phi0_v10.copy()
phi0_v11[4] *= 1.3
phi0_v11[5] *= 1.3

# Normalize rows to preserve approximate G₂ symmetry
row_norms = np.linalg.norm(phi0_v11, axis=1, keepdims=True) + 1e-8
phi0_v11 /= row_norms

print("φ⁰_v11 kernel:")
print(phi0_v11)
print("
Eigenvalues:", eigvalsh(phi0_v11))


In [ ]:

try:
    with open('training_patches.pkl','rb') as f:
        training_patches = pickle.load(f)
    with open('validation_patches.pkl','rb') as f:
        validation_patches = pickle.load(f)
    print('Loaded training and validation patches.')
except FileNotFoundError:
    raise FileNotFoundError('Upload "training_patches.pkl" and "validation_patches.pkl" using the Colab file uploader.')


In [ ]:

def cosine_sim(a,b):
    return float(np.dot(a.ravel(), b.ravel())/(np.linalg.norm(a)*np.linalg.norm(b) + 1e-8))

def score_patch(cov_mat, aspect_ratio=1.0):
    rotations = [np.rot90(cov_mat, k) for k in range(8)]
    base = max(cosine_sim(phi0_v11, r) for r in rotations)
    return base * np.exp(-0.5*(aspect_ratio-1.0)**2)


In [ ]:

results = []
for patch in validation_patches:
    cov = patch.get('cov') or patch.get('cov_matrix')
    if cov is None and 'elevation' in patch:
        data = np.array(patch['elevation'])
        cov = np.cov(data.reshape(-1, data.shape[-1]), rowvar=False)
    if cov is None:
        continue
    score = score_patch(np.array(cov), patch.get('aspect_ratio',1.0))
    results.append({
        'name': patch.get('name','unnamed'),
        'is_positive': patch.get('is_positive', True),
        'score': score,
        'features': np.array(cov)
    })

scores = np.array([r['score'] for r in results])
labels = np.array([1 if r['is_positive'] else 0 for r in results])
pos_scores = scores[labels==1]
neg_scores = scores[labels==0]
print(f'Positive mean: {pos_scores.mean():.4f} ± {pos_scores.std():.4f}')
print(f'Negative mean: {neg_scores.mean():.4f} ± {neg_scores.std():.4f}')


In [ ]:

threshold = (pos_scores.mean() + neg_scores.mean())/2
cohen_d = (pos_scores.mean()-neg_scores.mean())/np.sqrt(0.5*(pos_scores.var()+neg_scores.var()))
roc_fpr, roc_tpr, _ = roc_curve(labels, scores)
roc_auc = auc(roc_fpr, roc_tpr)

delta = pos_scores.mean() - neg_scores.mean()

plt.figure(figsize=(6,4))
plt.hist(pos_scores, bins=20, alpha=0.5, label='windmill')
plt.hist(neg_scores, bins=20, alpha=0.5, label='control')
plt.legend()
plt.xlabel('Score')
plt.ylabel('Count')
plt.title('Score Distribution')
plt.show()

plt.figure(figsize=(5,5))
plt.plot(roc_fpr, roc_tpr, label=f'AUC={roc_auc:.3f}')
plt.plot([0,1],[0,1],'--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend()
plt.title('ROC Curve')
plt.show()

names = [r['name'] for r in results]
bar_colors = ['tab:blue' if r['is_positive'] else 'tab:orange' for r in results]
plt.figure(figsize=(8,3))
plt.bar(range(len(results)), scores, color=bar_colors)
plt.xticks(range(len(results)), names, rotation=90)
plt.ylabel('Score')
plt.title('Individual Site Scores')
plt.tight_layout()
plt.show()

plt.figure(figsize=(4,3))
plt.imshow(phi0_v11, cmap='viridis')
plt.colorbar(label='weight')
plt.title('φ⁰ v11 Heatmap')
plt.show()

best_windmill = max((r for r in results if r['is_positive']), key=lambda x: x['score'])
worst_windmill = min((r for r in results if r['is_positive']), key=lambda x: x['score'])
best_control = max((r for r in results if not r['is_positive']), key=lambda x: x['score'])

plt.figure(figsize=(4,3))
plt.imshow(best_windmill['features'], cmap='gray')
plt.title('Best Windmill')
plt.show()

plt.figure(figsize=(4,3))
plt.imshow(worst_windmill['features'], cmap='gray')
plt.title('Worst Windmill')
plt.show()

plt.figure(figsize=(4,3))
plt.imshow(best_control['features'], cmap='gray')
plt.title('Best Control')
plt.show()

print(f"Mean±Std Pos: {pos_scores.mean():.4f} ± {pos_scores.std():.4f}")
print(f"Mean±Std Neg: {neg_scores.mean():.4f} ± {neg_scores.std():.4f}")
print(f"Cohen's d: {cohen_d:.4f}")
print(f"ROC AUC: {roc_auc:.4f}")
print(f"Threshold: {threshold:.4f}")
print(f"Δ: {delta:.4f}")


In [ ]:

np.save('phi0_kernel_v11_optimized.npy', phi0_v11)
plt.figure()
plt.plot(roc_fpr, roc_tpr, label=f'AUC={roc_auc:.3f}')
plt.plot([0,1],[0,1],'--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend()
plt.title('ROC Curve')
plt.savefig('kernel_validation_auc_v11.png')
plt.close()
print('Saved phi0_kernel_v11_optimized.npy and kernel_validation_auc_v11.png')
